# Introduction to Patent_Client

The patent_client package provides easy access to a number of USPTO and EPO patent data resources, all through a familiar ORM-like interface. If you have experience with the [Django ORM](https://docs.djangoproject.com/en/2.1/topics/db/queries/#retrieving-objects) with some extra features to accomodate the various data sources. Each data source is encapsulated in a Model object that represents various API's. The top-level models are:

- `USApplication` (USPTO Patent Examination Data System)
- `PtabTrial` (USPTO PTAB Trials API)
- `Assignment` (USPTO Assignments API)
- `Inpadoc` (EPO Inpadoc API)
- `Epo` (EPO Register API)
- `ITCInvestigation` (USITC EDS API)

Several of these models also have sub-models, as discussed below. This notebook shows some simple examples for doing data lookups with each of the models. The other notebooks in the repository show more complex analysis using the [Pandas](https://pandas.pydata.org/) data science library.

## USApplication (USPTO Patent Examination Data System)

The first model we'll look at is USApplication, an abstraction on top of the [USPTO Patent Examination Data System API](https://ped.uspto.gov/peds/#/apiDocumentation). The first step, which is common to all models, is to import the model object - USApplication. 

In [1]:
from patent_client import USApplication

The model can then be queried to get actual data back to do analysis, review, etc. All queries take one of two forms:

- Model.objects.filter(query) -> retreives many objects
- Model.objects.get(query) -> retreives one object, otherwise throws an error

For example, we can query USApplications by applicant by passing the query:
```python
    USApplication.objects.filter(first_named_applicant='Tesla Motors')
```

This query will retrieve all applications that name Tesla Motors as an applicant. By default, if you pass in only an argument without a keyword, like this:

```python
    USApplication.objects.get("10123456")
```
the model assumes that you're referring to the application number, and fetches the corresponding app. A complete list of available query fields is available at Model.objects.query_fields. For example:

```python
    USApplication.objects.query_fields
```

*NOTE:* If you ever need help with something, try calling the Python builtin "help" function. All models have (or should have) super-handy help text about queries, available information, related objects, etc. E.g.:

```python
    help(USApplication)
```  

To protect the innocent, we'll use Tesla's patent portfolio as an example - a company that has publicly agreed not to assert its patents against others. To do a lookup by applicant, we query USApplication object like this:

In [2]:
tesla_apps = USApplication.objects.filter(first_named_applicant='Tesla Motors')
print('We found apps with these applicant names:')
print(list(set(a.applicants[0].name for a in tesla_apps)))

We found apps with these applicant names:
['Inc.\nTesla Motors,', 'TESLA MOTORS, INC.', 'Tesla Motors, Inc.', 'TESLA MOTORS, INC', 'Tesla Motors Canada ULC', 'TESLA MOTORS. INC.']


Let's grab one case, and explore the data therein. Note that all Model objects are dataclasses, with all the magic that includes, including descriptive __str__ functions with basic data:

In [3]:
tesla_app = tesla_apps[15]
tesla_app

USApplication(appl_id='15016202', patent_title='PYROTECHNIC DISCONNECT WITH ARC SPLITTER PLATES', app_status='Patented Case')

A complete list of data attributes can be found by examining the fields of the dataclass

In [4]:
[f.name for f in tesla_app.fields()]

['appl_id',
 'patent_title',
 'app_status',
 'inventors',
 'app_filing_date',
 'app_location',
 'first_inventor_file',
 'app_type',
 'app_entity_status',
 'app_confr_number',
 'applicants',
 'app_status_date',
 'app_cls_sub_cls',
 'app_grp_art_number',
 'corr_addr_cust_no',
 'app_cust_number',
 'app_attr_dock_number',
 'patent_number',
 'patent_issue_date',
 'app_early_pub_number',
 'app_early_pub_date',
 'app_exam_name',
 'wipo_early_pub_number',
 'wipo_early_pub_date',
 'transactions',
 'child_continuity',
 'parent_continuity',
 'pta_pte_tran_history',
 'pta_pte_summary',
 'correspondent',
 'attorneys']

As you can see, there is a wealth of data about this particular case. Most of the names are relatively self-explanatory. We can look at some basic data by grabbing all the elements that start with "app". First, we'll use the .as_dict() method to convert the application to a standard Python dictionary. Then we'll only print items if they contain "_app":

In [5]:
for k, v in tesla_app.as_dict().items():
    if 'app_' in k or k == 'appl_id':
        print(f'{k}: {v}')
        

app_attr_dock_number: TSLA.410A1
app_cls_sub_cls: 337/414000
app_confr_number: 7035
app_cust_number: 113241
app_early_pub_date: 2017-08-10
app_early_pub_number: US20170229268A1
app_entity_status: UNDISCOUNTED
app_exam_name: CRUM, JACOB R
app_filing_date: 2016-02-04
app_grp_art_number: 2835
app_location: ELECTRONIC
app_status: Patented Case
app_status_date: 2019-09-04
app_type: Utility
appl_id: 15016202


There are also some composite data elements. These elements consist of several related Model objects that model some part of the Application data. These data elements are:
    
- `USApplication.transactions` (a list of Transaction objects)
- `USApplication.children` and `USApplication.parents` (both are lists of Relationship objects)
- `USApplication.foreign_priority` (a list of ForeignPriority objects)
- `USApplication.pta_pte_history` (a list of PtaPteHistory objects)
- `USApplication.pta_pte_summary` (a PtaPteSummary object)
- `USApplication.correspondent` (a Correspondent object)
- `USApplication.attorneys` (a list of Attorney objects)
    
To use as an example, let's take a look at the transaction data, and fetch the first 15 records. 

In [6]:
tesla_app.transactions[:15]

[Transaction(date=datetime.date(2019, 9, 24), code='PGM/', description='Recordation of Patent Grant Mailed'),
 Transaction(date=datetime.date(2019, 9, 5), code='EML_NTR', description='Email Notification'),
 Transaction(date=datetime.date(2019, 9, 4), code='WPIR', description='Issue Notification Mailed'),
 Transaction(date=datetime.date(2019, 9, 24), code='PTAC', description='Patent Issue Date Used in PTA Calculation'),
 Transaction(date=datetime.date(2019, 8, 21), code='D1935', description='Dispatch to FDC'),
 Transaction(date=datetime.date(2019, 8, 13), code='REAS', description='Response to Reasons for Allowance'),
 Transaction(date=datetime.date(2019, 8, 15), code='PILS', description='Application Is Considered Ready for Issue'),
 Transaction(date=datetime.date(2019, 8, 13), code='N084', description='Issue Fee Payment Verified'),
 Transaction(date=datetime.date(2019, 8, 13), code='IFEE', description='Issue Fee Payment Received'),
 Transaction(date=datetime.date(2019, 5, 22), code='ELC

You might notice that the above data is relatively messy. We can make the data look a bit cleaner by converting the objects to a Pandas dataframe. Every list of objects in patent_client has a .to_pandas method that will convert it to a pandas dataframe:

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', -1) # makes sure the table runs all the way to the edge

tesla_app.transactions[:15].to_pandas()

/Users/parkerhancock/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,code,date,description
0,PGM/,2019-09-24,Recordation of Patent Grant Mailed
1,EML_NTR,2019-09-05,Email Notification
2,WPIR,2019-09-04,Issue Notification Mailed
3,PTAC,2019-09-24,Patent Issue Date Used in PTA Calculation
4,D1935,2019-08-21,Dispatch to FDC
5,REAS,2019-08-13,Response to Reasons for Allowance
6,PILS,2019-08-15,Application Is Considered Ready for Issue
7,N084,2019-08-13,Issue Fee Payment Verified
8,IFEE,2019-08-13,Issue Fee Payment Received
9,ELC_RVW,2019-05-22,Electronic Review


## PtabTrial (USPTO PTAB Api)

Now that you've seen a little bit of the USApplication API, let's take a look at the PTAB Trials API. We'll start with the usual import:

In [8]:
from patent_client import PtabProceeding

For randomness' sake, we'll pick the first IPR of 2017 to play around with, and look it up by its trial number:

In [9]:
trial = PtabProceeding.objects.get('IPR2017-00001')
print(f'PTAB Trial No. {trial.trial_number} was filed {trial.filing_date} to challenge U.S. Patent {trial.patent_number}')
print(f'The case is styled {trial.petitioner_party_name} v. {trial.patent_owner_name}')

TypeError: __init__() missing 1 required positional argument: 'respondent_party_name'

The complete list of available data is again available at .attrs:

In [9]:
print(trial.attrs)

['trial_number', 'application_number', 'patent_number', 'petitioner_party_name', 'patent_owner_name', 'inventor_name', 'prosecution_status', 'filing_date', 'accorded_filing_date', 'institution_decision_date', 'last_modified_datetime', 'documents']


Where possible, there are also related objects that can be fetch from a Model object. For example, the challenged patent has a related USApplication. A complete list of all related objects and other data can be found by calling the help function on the object:

In [10]:
help(trial)

Help on PtabTrial in module patent_client.uspto_ptab object:

class PtabTrial(patent_client.util.Model)
 |  Ptab Trial
 |  ==========
 |  This object wraps the PTAB's public API (https://ptabdata.uspto.gov)
 |  
 |  ---------------------
 |  To Fetch a PTAB Trial
 |  ---------------------
 |  
 |  The main way to create a PtabTrial is by querying the PtabTrial manager at PtabTrial.objects
 |  
 |      PtabTrial.objects.filter(query) -> obtains multiple matching applications
 |      PtabTrial.objects.get(query) -> obtains a single matching application, errors if more than one is retreived
 |  
 |  The query can either be a single number, which is treated as a trial number, or a keyword argument:
 |  
 |      PtabTrial.objects.get('IPR2016-00831') -> Retreives a single trial
 |      PtabTrial.objects.filter(patent_number='6103599') -> retreives all PTAB trials involving US Patent Number 6103599
 |  
 |  A complete list of query fields is available at PtabTrial.objects.query_fields
 |  
 

From the data above, we see that there is a related data object called "us_application". That is, we can fetch the application object that corresponds to the PtabTrial simply by retreiving the attribute .us_application, like so:

In [11]:
trial_app = trial.us_application
print(trial_app)
print({k:v for (k, v) in trial_app.as_dict().items() if 'app_' in k})

<USApplication(appl_id=11395685)>
{'app_filing_date': datetime.date(2006, 3, 31), 'app_exam_name': 'PHAM, TOAN NGOC', 'app_early_pub_number': 'US20060181406A1', 'app_early_pub_date': datetime.date(2006, 8, 17), 'app_location': 'ELECTRONIC', 'app_grp_art_number': '2612', 'app_status': 'Application Involved in Court Proceedings', 'app_status_date': datetime.date(2018, 5, 16), 'app_attr_dock_number': 'STAT3', 'app_type': 'Utility', 'app_cust_number': '89818', 'app_cls_sub_cls': '340/531000', 'app_entity_status': 'UNDISCOUNTED', 'app_confr_number': '1922'}
